<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Algorithm-1.2" data-toc-modified-id="Algorithm-1.2-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Algorithm 1.2</a></span></li><li><span><a href="#Algorithm-1.3" data-toc-modified-id="Algorithm-1.3-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Algorithm 1.3</a></span></li><li><span><a href="#Algorithm-1.4" data-toc-modified-id="Algorithm-1.4-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Algorithm 1.4</a></span></li><li><span><a href="#Algorithm-1.5" data-toc-modified-id="Algorithm-1.5-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Algorithm 1.5</a></span></li></ul></div>

## Algorithm 1.2

In [3]:
import numpy as np

from matplotlib import cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from pylab import rcParams
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['font.family'] = 'serif'
# mpl.rcParams['font.serif'] = ['Helvetica']

# plt.style.use('seaborn-white') # use sans-serif fonts

rcParams['figure.figsize'] = 5, 5

label_size = 10
mpl.rcParams['xtick.labelsize'] = label_size
mpl.rcParams['ytick.labelsize'] = label_size
mpl.rcParams['axes.labelsize'] = 15

mpl.rcParams['axes.spines.left'] = True   ## display axis spines
mpl.rcParams['axes.spines.bottom'] = True
mpl.rcParams['axes.spines.top'] = True
mpl.rcParams['axes.spines.right'] = True
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.direction'] = 'out'
mpl.rcParams['ytick.direction'] = 'out'
mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 1.0
mpl.rcParams['ytick.major.width'] = 1.0


In [4]:
## Define time and space discretizations

xi = np.linspace(-10, 10, 400)
ti = np.linspace(0, 4*np.pi, 200)
dt = ti[1] - ti[0]
[xgrid, tgrid] = np.meshgrid(xi,ti)


## Create two spatio-temporal patterns
f1 = (1/np.cosh(xgrid + 3))*(1*np.exp(1j*2.3*tgrid))
f2 = ((1/np.cosh(xgrid))*np.tanh(xgrid))*(2*np.exp(1j*2.8*tgrid))

## Combine signals and make data matrix
f = f1 + f2
X = f.T

In [165]:
%matplotlib

fig = plt.figure(figsize = (8,8))
ax_f1 = fig.add_subplot(221, projection='3d')
ax_f2 = fig.add_subplot(222, projection='3d')
ax_f = fig.add_subplot(223, projection='3d')

ax_f1.plot_surface(xgrid.T, tgrid.T, np.real(f1.T), \
                   antialiased = True, cmap = 'gray_r', \
                   linewidth = 0, alpha = 0.6)
ax_f1.view_init(30,-50)

ax_f2.plot_surface(xgrid.T, tgrid.T, np.real(f2.T), \
                   antialiased = True, cmap = 'gray_r', \
                   linewidth = 0, alpha = 0.6)
ax_f2.view_init(30,-50)

ax_f.plot_surface(xgrid.T, tgrid.T, np.real(X), \
                  antialiased = True, cmap = 'gray_r', \
                  linewidth = 0, alpha = 0.6)
ax_f.view_init(30,-50)

# plt.savefig('test.pdf', bbox_inches = 'tight')
plt.show()



Using matplotlib backend: MacOSX


## Algorithm 1.3

In [5]:
from numpy.linalg import svd, inv, eig

## Create DMD data matrices
X1 = X[:, 0:-1]
X2 = X[:, 1:]

## SVD and rank-2 truncation
r = 2 # rank truncation
[U, S, V] = svd(X1, full_matrices = False)
S = np.diagflat(S) # because numpy returns S as 1D array
Ur = U[:, 0:r] 
Sr = S[0:r, 0:r]
Vr = V[:, 0:r]


In [123]:
U.shape, S.shape, V.shape
Ur.shape, Sr.shape, Vr.shape

((400, 2), (2, 2), (199, 2))

In [124]:
## Build Atilde and DMD Modes
Atilde = np.dot(Ur.T,np.dot(X2,np.dot(Vr,inv(Sr))))
[W, D] = eig(Atilde)
W = np.diagflat(W)
Phi = np.dot(X2,np.dot(np.dot(Vr,inv(Sr)),W));  # DMD Modes

In [125]:
Atilde.shape, W.shape, D.shape, Phi.shape

((2, 2), (2, 2), (2, 2), (400, 2))

In [126]:
## DMD Spectra
lambda_vec = np.diag(D)
omega = np.log(lambda_vec)/dt

In [159]:
import sys
## Compute DMD Solution
x1 = np.array(X[:, 0])
b = np.dot(np.linalg.pinv(Phi),x1)
time_dynamics = np.zeros((r,np.size(ti)))

for itr in range(np.size(ti)):
    time_dynamics[:,itr] = b*np.exp(omega*ti[itr])
    
X_dmd = np.dot(Phi,time_dynamics)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [160]:
X_dmd.shape
# np.iscomplex(X_dmd)

(400, 200)

In [168]:
omega
# something fishy going on earlier.

array([-1.09599859+0.j, -1.05913917+0.j])

In [166]:

ax_dmd_f = fig.add_subplot(224, projection = '3d')

ax_dmd_f.plot_surface(xgrid.T, tgrid.T, np.real(X_dmd), \
                      antialiased = True, cmap = 'gray_r', \
                      linewidth = 0, alpha = 0.6)
ax_dmd_f.view_init(30,-50)
plt.show()


## Algorithm 1.4

In [144]:
[U, S, V] = svd(X, full_matrices = False)
S = np.diagflat(S) # because numpy returns S as 1D array);
pc1 = U[:, 0] # first PCA mode
pc2 = U[:, 1] # second PCA mode
time_pc1 = V[:, 0] # temporal evolution of pc1
time_pc2 = V[:, 1] # temporal evolution of pc2

## Algorithm 1.5

In [150]:
# Requires a fast ICA software package
from sklearn.decomposition import FastICA, PCA

IC = FastICA(n_components=10)
S_ = IC.fit_transform(X)  # Reconstruct signals
ICt = IC.mixing_  # Get estimated mixing matrix

ic1 = IC[0,:] # first ICA mode
ic2 = IC[1, :] # second ICA mode
time_ic1 = ICt[:,0] # temporal evolution of ic1
time_ic2 = ICt[:,1] # temporal evolution of ic2


# [IC, ICt, ~] = fastica(real(X)');


/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)
/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:118: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


TypeError: 'FastICA' object is not subscriptable